Definição do problema
Os orçamentos públicos dos entes federativos prevêem a projeção de receitas, e a partir delas são planejadas as despesas para o ano.
É sempre um desafio para os gestores públicos prever os valores realizáveis dessas receitas, dadas as variáveis que afetam a arrecadação.
Este trabalho pretende apresentar um modelo supervisionado que auxilie a projeção das receitas tributárias para os anos seguintes, baseado no histórico de arrecadação dos tributos, por estado.
Nossa hipótese inicial é de que perfil do crescimento ou redução de cada tributo ao passar dos anos possa indicar os efeitos na arrecadação dos anos seguintes.
Para isso, será utilizado um dataset extraído do portal de dados abertos do governo federal. Partimos do pressuposto de que as informações nele contidas, foram aprovados pelos tribunais de contas e que são os dados oficiais de cada estado.
Os dados do dataset estão atualizados até março de 2023, mas dada a proposta de projeção de receitas para o ano, serão considerados apenas os dados até dezembro de 2022. Em virtude da pandemia e os seus efeitos econômicos, o comportamento dos dados de 2020 e 2021 farão parte do teste, porém poderão serem avaliados em separado.

In [19]:
#Importando bibliotecas que serão utilizadas no trabalho
import pandas as pd
import matplotlib.pyplot as plt

Carregando dataset

In [20]:
#Definindo path do dataset
url_dados: str = 'https://raw.githubusercontent.com/leonardevop/mvp_1/main/arrecadacao-estado%20(1).csv'

#Carregando dataset. Utilizado encoding cp1252 após inconsistências com o padrão utf-8.
arrecadacao = pd.read_csv(url_dados, delimiter=';', encoding='cp1252')

Análise de Dados

In [21]:
#Avaliando o conjunto completo do dataframe
arrecadacao

,Ano,Mês,UF,IMPOSTO SOBRE IMPORTAÇÃO,IMPOSTO SOBRE EXPORTAÇÃO,IPI - FUMO,IPI - BEBIDAS,IPI - AUTOMÓVEIS,IPI - VINCULADO À IMPORTACAO,IPI - OUTROS,...,REFIS,PAES,"RETENÇÃO NA FONTE - LEI 10.833, Art. 30",PAGAMENTO UNIFICADO,OUTRAS RECEITAS ADMINISTRADAS,DEMAIS RECEITAS,RECEITA PREVIDENCIÁRIA,RECEITA PREVIDENCIÁRIA - PRÓPRIA,RECEITA PREVIDENCIÁRIA - DEMAIS,ADMINISTRADAS POR OUTROS ÓRGÃOS
0,2000,Janeiro,AC,231,0,292096,0,0,167,1558,...,NaN,NaN,NaN,NaN,20911,NaN,NaN,NaN,NaN,NaN
1,2000,Janeiro,AL,475088,33873,1329338,812470,0,141735,3676847,...,NaN,NaN,NaN,NaN,127189,NaN,NaN,NaN,NaN,NaN
2,2000,Janeiro,AM,11679405,0,1507146,1791471,27796,4414483,1800346,...,NaN,NaN,NaN,NaN,187988,NaN,NaN,NaN,NaN,NaN
3,2000,Janeiro,AP,24267,0,424862,1419,321,22333,12165,...,NaN,NaN,NaN,NaN,10769,NaN,NaN,NaN,NaN,NaN
4,2000,Janeiro,BA,10476457,13,5542123,11533707,0,7496476,23743858,...,NaN,NaN,NaN,NaN,536498,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7528,2023,Março,RS,"158362,17",0,0,0,0,"24208,66","235246,06",...,NaN,NaN,NaN,NaN,"562940,66",NaN,"68746002,94",NaN,NaN,"1065145,948"
7529,2023,Março,SC,1002120548,"8585,32",0,"8975324,12","30117458,81","388110295,6","372068279,7",...,NaN,NaN,NaN,NaN,"89713060,55",NaN,2420498485,NaN,NaN,"29070566,29"
7530,2023,Março,SE,1792437623,"-13094,21","2318037,76","45180880,2","183927349,9","645108876,4",1033460741,...,NaN,NaN,NaN,NaN,1480417865,NaN,17794414313,NaN,NaN,257433219
7531,2023,Março,SP,"3794400,24","21469841,3","374508,35","2039949,68",0,"1113987,7","6576891,62",...,NaN,NaN,NaN,NaN,"3805839,03",NaN,"234773889,8",NaN,NaN,"3542186,066"


In [22]:
#Verificando os tipos de dados de cada coluna do dataframe
arrecadacao.dtypes
#Dado que o dataset está voltado para a apresentação de valores de cada fonte de arrecadação, percebe-se algo estranho com a maioria das colunas apresentadas como objects.

Ano                                                               int64
Mês                                                              object
UF                                                               object
IMPOSTO SOBRE IMPORTAÇÃO                                         object
IMPOSTO SOBRE EXPORTAÇÃO                                         object
IPI - FUMO                                                       object
IPI - BEBIDAS                                                    object
IPI - AUTOMÓVEIS                                                 object
IPI - VINCULADO À IMPORTACAO                                     object
IPI - OUTROS                                                     object
IRPF                                                             object
IRPJ - ENTIDADES FINANCEIRAS                                     object
IRPJ - DEMAIS EMPRESAS                                           object
IRRF - RENDIMENTOS DO TRABALHO                                  

In [23]:
#Consultando os primeiros registros do dataframe
arrecadacao.head()
#Identificados alguns dados faltantes com o registro "NaN".
#Identificado que o dataset possui 45 atributos.

,Ano,Mês,UF,IMPOSTO SOBRE IMPORTAÇÃO,IMPOSTO SOBRE EXPORTAÇÃO,IPI - FUMO,IPI - BEBIDAS,IPI - AUTOMÓVEIS,IPI - VINCULADO À IMPORTACAO,IPI - OUTROS,...,REFIS,PAES,"RETENÇÃO NA FONTE - LEI 10.833, Art. 30",PAGAMENTO UNIFICADO,OUTRAS RECEITAS ADMINISTRADAS,DEMAIS RECEITAS,RECEITA PREVIDENCIÁRIA,RECEITA PREVIDENCIÁRIA - PRÓPRIA,RECEITA PREVIDENCIÁRIA - DEMAIS,ADMINISTRADAS POR OUTROS ÓRGÃOS
0,2000,Janeiro,AC,231,0,292096,0,0,167,1558,...,NaN,NaN,NaN,NaN,20911,NaN,NaN,NaN,NaN,NaN
1,2000,Janeiro,AL,475088,33873,1329338,812470,0,141735,3676847,...,NaN,NaN,NaN,NaN,127189,NaN,NaN,NaN,NaN,NaN
2,2000,Janeiro,AM,11679405,0,1507146,1791471,27796,4414483,1800346,...,NaN,NaN,NaN,NaN,187988,NaN,NaN,NaN,NaN,NaN
3,2000,Janeiro,AP,24267,0,424862,1419,321,22333,12165,...,NaN,NaN,NaN,NaN,10769,NaN,NaN,NaN,NaN,NaN
4,2000,Janeiro,BA,10476457,13,5542123,11533707,0,7496476,23743858,...,NaN,NaN,NaN,NaN,536498,NaN,NaN,NaN,NaN,NaN


In [24]:
#Consultando os últimos registros no dataframe
arrecadacao.tail()
#Identificado que os últimos registros indicam que os dados foram atualizados até Março de 2023.
#Verificado que há 7533 linhas no dataset.

,Ano,Mês,UF,IMPOSTO SOBRE IMPORTAÇÃO,IMPOSTO SOBRE EXPORTAÇÃO,IPI - FUMO,IPI - BEBIDAS,IPI - AUTOMÓVEIS,IPI - VINCULADO À IMPORTACAO,IPI - OUTROS,...,REFIS,PAES,"RETENÇÃO NA FONTE - LEI 10.833, Art. 30",PAGAMENTO UNIFICADO,OUTRAS RECEITAS ADMINISTRADAS,DEMAIS RECEITAS,RECEITA PREVIDENCIÁRIA,RECEITA PREVIDENCIÁRIA - PRÓPRIA,RECEITA PREVIDENCIÁRIA - DEMAIS,ADMINISTRADAS POR OUTROS ÓRGÃOS
7528,2023,Março,RS,"158362,17",0,0,0,0,"24208,66","235246,06",...,NaN,NaN,NaN,NaN,"562940,66",NaN,"68746002,94",NaN,NaN,"1065145,948"
7529,2023,Março,SC,1002120548,"8585,32",0,"8975324,12","30117458,81","388110295,6","372068279,7",...,NaN,NaN,NaN,NaN,"89713060,55",NaN,2420498485,NaN,NaN,"29070566,29"
7530,2023,Março,SE,1792437623,"-13094,21","2318037,76","45180880,2","183927349,9","645108876,4",1033460741,...,NaN,NaN,NaN,NaN,1480417865,NaN,17794414313,NaN,NaN,257433219
7531,2023,Março,SP,"3794400,24","21469841,3","374508,35","2039949,68",0,"1113987,7","6576891,62",...,NaN,NaN,NaN,NaN,"3805839,03",NaN,"234773889,8",NaN,NaN,"3542186,066"
7532,2023,Março,TO,"2465906,4",0,0,"323312,97",0,"1682315,41","1401138,66",...,NaN,NaN,NaN,NaN,"2334849,7",NaN,"195218428,8",NaN,NaN,"2955794,943"


In [25]:
#Consultando as 45 colunas do dataframe
arrecadacao.columns
#Identificado que há segregações do mesmo imposto em diferentes categorias.
#Identificadas colunas que podem conter subtotais das colunas segregadas, ou mudanças de critérios de totalização dos valores ao longo do tempo.

Index(['Ano', 'Mês', 'UF', 'IMPOSTO SOBRE IMPORTAÇÃO',
       'IMPOSTO SOBRE EXPORTAÇÃO', 'IPI - FUMO', 'IPI - BEBIDAS',
       'IPI - AUTOMÓVEIS', 'IPI - VINCULADO À IMPORTACAO', 'IPI - OUTROS',
       'IRPF', 'IRPJ - ENTIDADES FINANCEIRAS', 'IRPJ - DEMAIS EMPRESAS',
       'IRRF - RENDIMENTOS DO TRABALHO', 'IRRF - RENDIMENTOS DO CAPITAL',
       'IRRF - REMESSAS P/ EXTERIOR', 'IRRF - OUTROS RENDIMENTOS',
       'IMPOSTO S/ OPERAÇÕES FINANCEIRAS', 'IMPOSTO TERRITORIAL RURAL',
       'IMPOSTO PROVIS.S/ MOVIMENT. FINANC. - IPMF', 'CPMF', 'COFINS',
       'COFINS - FINANCEIRAS', 'COFINS - DEMAIS',
       'CONTRIBUIÇÃO PARA O PIS/PASEP',
       'CONTRIBUIÇÃO PARA O PIS/PASEP - FINANCEIRAS',
       'CONTRIBUIÇÃO PARA O PIS/PASEP - DEMAIS', 'CSLL', 'CSLL - FINANCEIRAS',
       'CSLL - DEMAIS', 'CIDE-COMBUSTÍVEIS (parc. não dedutível)',
       'CIDE-COMBUSTÍVEIS', 'CONTRIBUIÇÃO PLANO SEG. SOC. SERVIDORES',
       'CPSSS - Contrib. p/ o Plano de Segurid. Social Serv. Público',
       'CONTRIB

In [26]:
#Confirmando dimensões do dataframe
arrecadacao.shape
#Volume de dados anteriormente identificados confirmados

(7533, 45)

In [27]:
#Verificando o total de registros por cada coluna do dataframe
arrecadacao.count()
#Identificado que apenas algumas colunas possuem registros em todas as linhas, sugerindo eventuais erros ou alterações de critérios de totalização. 

Ano                                                             7533
Mês                                                             7533
UF                                                              7533
IMPOSTO SOBRE IMPORTAÇÃO                                        7533
IMPOSTO SOBRE EXPORTAÇÃO                                        7533
IPI - FUMO                                                      7533
IPI - BEBIDAS                                                   7533
IPI - AUTOMÓVEIS                                                7533
IPI - VINCULADO À IMPORTACAO                                    7533
IPI - OUTROS                                                    7533
IRPF                                                            7533
IRPJ - ENTIDADES FINANCEIRAS                                    7533
IRPJ - DEMAIS EMPRESAS                                          7533
IRRF - RENDIMENTOS DO TRABALHO                                  7533
IRRF - RENDIMENTOS DO CAPITAL     

In [28]:
#Obtendo um resumo das informações do dataframe
arrecadacao.describe()
#Verificado que tratamento das colunas de data estão gerando cálculos indevidos

,Ano,IMPOSTO PROVIS.S/ MOVIMENT. FINANC. - IPMF,REFIS,PAES,"RETENÇÃO NA FONTE - LEI 10.833, Art. 30",PAGAMENTO UNIFICADO
count,7533.000000,648.0,3.240000e+02,3.240000e+02,2.700000e+02,3.780000e+02
mean,2011.129032,0.0,2.163363e+06,8.472560e+06,1.295935e+07,9.749743e+06
std,6.713017,0.0,3.904852e+06,1.657378e+07,3.508999e+07,1.876911e+07
min,2000.000000,0.0,-1.797889e+06,1.360000e+05,9.309000e+03,1.161950e+05
25%,2005.000000,0.0,2.972922e+05,1.179268e+06,5.177198e+05,1.236024e+06
50%,2011.000000,0.0,7.620455e+05,2.561976e+06,2.529214e+06,2.937894e+06
75%,2017.000000,0.0,2.336568e+06,6.589126e+06,8.767503e+06,8.698714e+06
max,2023.000000,0.0,2.458967e+07,8.532303e+07,2.138863e+08,1.032222e+08


In [29]:
#Alterando a ordem da distribuição por UF, Ano e Mês
arrecadacao.sort_values(by=['UF', 'Ano', 'Mês'])
#Identificado que o atributo mês, por ser categórico, atrapalha a ordenação


,Ano,Mês,UF,IMPOSTO SOBRE IMPORTAÇÃO,IMPOSTO SOBRE EXPORTAÇÃO,IPI - FUMO,IPI - BEBIDAS,IPI - AUTOMÓVEIS,IPI - VINCULADO À IMPORTACAO,IPI - OUTROS,...,REFIS,PAES,"RETENÇÃO NA FONTE - LEI 10.833, Art. 30",PAGAMENTO UNIFICADO,OUTRAS RECEITAS ADMINISTRADAS,DEMAIS RECEITAS,RECEITA PREVIDENCIÁRIA,RECEITA PREVIDENCIÁRIA - PRÓPRIA,RECEITA PREVIDENCIÁRIA - DEMAIS,ADMINISTRADAS POR OUTROS ÓRGÃOS
81,2000,Abril,AC,69,0,281844,0,0,53,15332,...,NaN,NaN,NaN,NaN,16011,NaN,NaN,NaN,NaN,NaN
189,2000,Agosto,AC,514,0,278217,0,0,0,12191,...,NaN,NaN,NaN,NaN,53755,NaN,NaN,NaN,NaN,NaN
297,2000,Dezembro,AC,784,0,313571,0,0,111,7409,...,NaN,NaN,NaN,NaN,38439,NaN,NaN,NaN,NaN,NaN
27,2000,Fevereiro,AC,441,0,255361,0,0,0,11437,...,NaN,NaN,NaN,NaN,21142,NaN,NaN,NaN,NaN,NaN
0,2000,Janeiro,AC,231,0,292096,0,0,167,1558,...,NaN,NaN,NaN,NaN,20911,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7397,2022,Outubro,TO,"0,35","493,47",0,"9131,83",0,0,"1889354,71",...,NaN,NaN,NaN,NaN,"3977329,12",NaN,193062055,NaN,NaN,"13374995,73"
7370,2022,Setembro,TO,"843,15","27,32",0,"180894,42",0,"489940,66","2368161,07",...,NaN,NaN,NaN,NaN,"5444961,68",NaN,"195586326,2",NaN,NaN,"3669632,027"
7505,2023,Fevereiro,TO,"3297891,78","-0,04",0,"363439,44","6862,02","1085845,77","977873,79",...,NaN,NaN,NaN,NaN,5169123,NaN,"183745422,6",NaN,NaN,"2698647,263"
7478,2023,Janeiro,TO,"3174124,04","-0,44",0,"376826,17",0,"1624854,21","1088926,04",...,NaN,NaN,NaN,NaN,"2072091,62",NaN,"204004654,6",NaN,NaN,"2675652,344"


In [30]:

#Verificando algumas colunas com segregações para checar se há subtotalizações, analisando todas as linhas.
arrecadacao.loc[:, ['IPI - FUMO', 'IPI - BEBIDAS', 'IPI - AUTOMÓVEIS', 'IPI - VINCULADO À IMPORTACAO', 'IPI - OUTROS']]
#Não identificadas subtotalizações, mas aparentemente, informações inconsistentes, tais como arrecadação zerada de IPI para Fumo e Bebidas.


,IPI - FUMO,IPI - BEBIDAS,IPI - AUTOMÓVEIS,IPI - VINCULADO À IMPORTACAO,IPI - OUTROS
0,292096,0,0,167,1558
1,1329338,812470,0,141735,3676847
2,1507146,1791471,27796,4414483,1800346
3,424862,1419,321,22333,12165
4,5542123,11533707,0,7496476,23743858
...,...,...,...,...,...
7528,0,0,0,"24208,66","235246,06"
7529,0,"8975324,12","30117458,81","388110295,6","372068279,7"
7530,"2318037,76","45180880,2","183927349,9","645108876,4",1033460741
7531,"374508,35","2039949,68",0,"1113987,7","6576891,62"


In [31]:
arrecadacao.loc[:,['Ano', 'Mês', 'UF','COFINS', 'COFINS - FINANCEIRAS', 'COFINS - DEMAIS']]
#Identificada que a distribuição inicial não contém números para as segregações, e as finais, contém. Provavelmente, houve uma mudança de critério de totalização em um determinado período, onde se passou a usar a segregação dos valores para os dados de COFINS.

,Ano,Mês,UF,COFINS,COFINS - FINANCEIRAS,COFINS - DEMAIS
0,2000,Janeiro,AC,1770475,NaN,NaN
1,2000,Janeiro,AL,7493570,NaN,NaN
2,2000,Janeiro,AM,44000981,NaN,NaN
3,2000,Janeiro,AP,1682102,NaN,NaN
4,2000,Janeiro,BA,81027229,NaN,NaN
...,...,...,...,...,...,...
7528,2023,Março,RS,NaN,"8803,35","16171548,73"
7529,2023,Março,SC,NaN,"2588785,15",2008500806
7530,2023,Março,SE,NaN,1295632231,9767005309
7531,2023,Março,SP,NaN,"2195048,99","60623099,81"


In [32]:
arrecadacao.loc[:,['Ano', 'Mês', 'UF','RECEITA PREVIDENCIÁRIA', 'RECEITA PREVIDENCIÁRIA - PRÓPRIA', 'RECEITA PREVIDENCIÁRIA - DEMAIS', 'ADMINISTRADAS POR OUTROS ÓRGÃOS']]


,Ano,Mês,UF,RECEITA PREVIDENCIÁRIA,RECEITA PREVIDENCIÁRIA - PRÓPRIA,RECEITA PREVIDENCIÁRIA - DEMAIS,ADMINISTRADAS POR OUTROS ÓRGÃOS
0,2000,Janeiro,AC,NaN,NaN,NaN,NaN
1,2000,Janeiro,AL,NaN,NaN,NaN,NaN
2,2000,Janeiro,AM,NaN,NaN,NaN,NaN
3,2000,Janeiro,AP,NaN,NaN,NaN,NaN
4,2000,Janeiro,BA,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7528,2023,Março,RS,"68746002,94",NaN,NaN,"1065145,948"
7529,2023,Março,SC,2420498485,NaN,NaN,"29070566,29"
7530,2023,Março,SE,17794414313,NaN,NaN,257433219
7531,2023,Março,SP,"234773889,8",NaN,NaN,"3542186,066"


In [33]:

arrecadacao.loc[:,['Ano', 'Mês', 'UF','CSLL', 'CSLL - FINANCEIRAS', 'CSLL - DEMAIS']]


,Ano,Mês,UF,CSLL,CSLL - FINANCEIRAS,CSLL - DEMAIS
0,2000,Janeiro,AC,392830,NaN,NaN
1,2000,Janeiro,AL,1775834,NaN,NaN
2,2000,Janeiro,AM,7652859,NaN,NaN
3,2000,Janeiro,AP,346964,NaN,NaN
4,2000,Janeiro,BA,15699282,NaN,NaN
...,...,...,...,...,...,...
7528,2023,Março,RS,NaN,"7853,5","7270770,66"
7529,2023,Março,SC,NaN,"3442155,54","277762977,8"
7530,2023,Março,SE,NaN,1886353611,2964038221
7531,2023,Março,SP,NaN,"2434271,36","27012833,41"


In [34]:
#Verificando as médias de valores
arrecadacao.groupby('UF').mean()
#Será necessário pré-processamento para analisar melhor os valores médios.

C:\Users\us6x\AppData\Local\Temp\ipykernel_19760\3818021024.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  arrecadacao.groupby('UF').mean()


,Ano,IMPOSTO PROVIS.S/ MOVIMENT. FINANC. - IPMF,REFIS,PAES,"RETENÇÃO NA FONTE - LEI 10.833, Art. 30",PAGAMENTO UNIFICADO
UF,,,,,,
AC,2011.129032,0.0,3.146183e+04,2.581838e+05,50946.3,2.744286e+05
AL,2011.129032,0.0,4.460020e+05,1.293728e+06,825347.6,1.449263e+06
AM,2011.129032,0.0,5.124104e+05,2.288855e+06,4546511.9,2.772890e+06
AP,2011.129032,0.0,-1.063942e+04,4.652649e+05,164126.0,5.720156e+05
BA,2011.129032,0.0,2.030807e+06,6.123181e+06,9447899.3,6.266294e+06
CE,2011.129032,0.0,1.771432e+06,3.877032e+06,3574198.1,5.576285e+06
DF,2011.129032,0.0,1.851827e+06,3.522120e+06,15114969.0,7.699806e+06
ES,2011.129032,0.0,8.370375e+05,3.901774e+06,5602276.7,4.543115e+06
GO,2011.129032,0.0,7.859777e+05,6.988374e+06,3142717.7,9.251319e+06
